## Задачи компьютерного зрения
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">



Кроме классификации CV решает и другие задачи.

Сегментация, детектирование и многое другое

## Формат разметки COCO

Прежде чем говорить о способах решения задач компьютерного зрения разберемся с форматами входных данных.

#### COCO - Common Objects in COntext

Один из наиболее популярных датасатов содержащий данные для сегментации и детектирования.

Содержит: 
- Категории
- Маски
- Ограничивающие боксы (*bounding boxes*)
- Описания (*captions*)
- Ключевые точки (*keypoints*)
- И многое другое

Загрузим датасет

In [ ]:
import requests, zipfile, io

r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Для работы с датасетом используется пакет `pycocotools`

[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Разберемся с форматом на примере одной записи

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  # Получаем ID котиков
print('ID класса "кот" = %i' % catIds[0])

imgIds = coco.getImgIds(catIds=catIds)  # Фильтруем датасет по тегу
print("Всего изображений с котами: %i" % len(imgIds))

Разберем что у нас в метаданных

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # Берем только первую картинку
img = img_list[0]
img

Посмотрим на изображение

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

Сконвертируем в PIL формат для удобства дальнейшей работы

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

#### Категории в COCO

Давайте посмотрим на примеры категорий в нашем датасете. Отобразим каждую 10ую категорию

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # Грузим категории
num2cat = {}  # Создаем словарь для хранения
print("Категории COCO: ")
for cat in cats:
    num2cat[cat["id"]] = cat["name"]
    if cat["id"] in range(0, 80, 10):
        print(cat["id"], ":", cat["name"], end="   \n")

В датасете так же есть категория **0**. Ее используют для обозначения класса фона.

Есть так же суперкатегории

In [ ]:
print(cats[2])
print(cats[3])

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\n".join(nms)))

#### Вернемся к метаданным

Помимо метаданных нам доступна разметка ([подробнее о разметке](!https://cocodataset.org/#format-data)), давайте ее загрузим и отобразим

In [ ]:
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)

Давайте разберем из чего состоит разметка

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])


dump_anns(anns)

Для объектов, которых слишком много существует отдельная метка `iscorwd`

In [ ]:
plt.rcParams["figure.figsize"] = (120, 60)

catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])

Как получить маску в виде массива?

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(4):
        msk = coco.annToMask(anns[i])
        ax[row, col].imshow(msk)
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        i += 1

А еще у нас есть bounding boxes

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)

RGB_img = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

for i in range(len(anns)):
    x, y, width, heigth = anns[i]["bbox"]
    x, y, width, heigth = int(x), int(y), int(width), int(heigth)
    if anns[i]["category_id"] == 1:
        color = (255, 255, 255)
    if anns[i]["category_id"] == 37:
        color = (255, 0, 0)
    if anns[i]["category_id"] == 40:
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x, y), (x + width, y + heigth), color, 2)
cv2_imshow(RGB_img)

И еще куча всего

#### Еще более глубокое понимание разметки

Что такое [run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)?

[Видео-разбор](https://www.youtube.com/watch?v=h6s61a_pqfM)

## Семантическая сегментация (*Semantic segmentation*)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-007.png" width="900">

Постановка задачи:

Предсказать класс для каждого пикселя.

Входные данные маска: 

[ x,y - > class_num ] 

Выходные данные маска:

[ x,y - > class_num ] 


###Способы решения



#### **a) Наивный.**

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-012.png" width="700">


Скользящим окном пройтись по изображению и предсказать клас для каждого пикселя с учетом его соседей.


#### **б) Разумный**

Убрать линейный слой в конце сети. По признакам во всех каналах определять класс каждого пикселя.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-015.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_15.png"  width="700">

В лекции №8 мы говорили о том что сверту 1x1 можно рассматривать как полносвязанный слой.


Именно так она и будет использоваться при сегментации.

Количество классов будет соответствовать числу каналов.


Проблемы:
- нужно большое рецептивное поле, следовательно много слоев ( L 3х3 conv -> 1+2L receptive field)
- очень медленно на полноразмерных картах активации

#### **в) Эффективный**


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-017.png" width="700">


Вернмся к традиционной структуре со сжатием пространственных размеров. А после нее добавим разжимающий блок. 

## Автокодировщик

Такая архитектура довольно популярна и применяется не только для сегментации: 


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-07.png" width="700">


- сглаживание шума;
- снижение размерности -> вектор признак
- генерация данных


Об этом будет целая отдельная лекция чуть позже


## Разжимающий (upsample) блок

### Изменение размеров изображений 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-08.png" width="700">


---


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/bilinear.png" width="300">

Билинейная интерполяция рассматривает квадрат 2x2 известных пикселя, окружающих неизвестный. В качестве интерполированного значения используется взвешенное усреднение этих четырёх пикселей.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/bicubic.png" width="300">


Бикубическая интерполяция идёт на один шаг дальше билинейной, рассматривая массив из 4x4 окружающих пикселей — всего 16. Поскольку они находятся на разных расстояниях от неизвестного пикселя, ближайшие пиксели получают при расчёте больший вес.





### Upsample в Pytorch
С картами признаков можно обращаться так же как и с пикселями. Для этого в Pytorch используется метод Upsample

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-11.png" width="700">

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt


def upsample(pil, mode="nearest", ax=ax):
    tensor = TF.to_tensor(pil)
    upsampler = nn.Upsample(scale_factor=2, mode=mode)
    tensor_128 = upsampler(tensor.unsqueeze(0))
    im_128 = TF.to_pil_image(tensor_128.squeeze()).convert("RGB")
    ax.imshow(im_128)
    ax.set_title(mode)
    ax.set_xlim(0, 20 * 2)
    ax.set_ylim(20 * 2, 0)


pic = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
pil_64 = pic.resize((64, 64))

fig, ax = plt.subplots(ncols=4, figsize=(15, 5))
ax[0].imshow(pil_64)
ax[0].set_title("Resized image")
ax[0].set_xlim(0, 20)
ax[0].set_ylim(20, 0)


upsample(pil_64, mode="nearest", ax=ax[1])
upsample(pil_64, mode="bilinear", ax=ax[2])
upsample(pil_64, mode="bicubic", ax=ax[3])

Обратите внимание что размер изображения увеличился в 2 раза!

### MaxUnpooling

Разница с предыдущим методом в том что индексы элементов запоминаются.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-019.png" width="700">

Сохраняем индексы каждого max pooling слоя.

При повышении разрешения копируем значения из выхода max pooling слоя с учетом запомненных индексов

[Документация к MaxPool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[Документация к MaxUnpool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-14.png" width="700">



In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt


def tensor_show(tensor, title="", ax=ax):
    im = TF.to_pil_image(tensor.squeeze()).convert("RGB")
    ax.set_title(title + str(im.size))
    ax.imshow(im)


pool = nn.MaxPool2d(kernel_size=2, return_indices=True)  # False by default
unpool = nn.MaxUnpool2d(kernel_size=2)

pil = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")

fig, ax = plt.subplots(ncols=5, figsize=(20, 5), sharex=True, sharey=True)

ax[0].set_title("original " + str(pil.size))
ax[0].imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Initial shape", tensor.shape)

# Downsample
tensor_half_res, indexes1 = pool(tensor)
print("Indexes shape", indexes1.shape)
tensor_show(tensor_half_res, "1/2 down", ax=ax[1])


tensor_q_res, indexes2 = pool(tensor_half_res)
tensor_show(tensor_q_res, "1/4 down", ax=ax[2])

# Upsample
tensor_half_res1 = unpool(tensor_q_res, indexes2)
tensor_show(tensor_half_res1, "1/2 up", ax=ax[3])

print(indexes1.shape)
tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show(tensor_recovered, "full size up", ax=ax[4])

Зачем нужен pad?


In [ ]:
import torch
import torch.nn.functional as F
import seaborn as sns

fig, ax = plt.subplots(ncols=2, figsize=(10, 5), sharex=True, sharey=True)
array = torch.ones((24, 24), dtype=int)
sns.heatmap(array, annot=True, fmt="d", ax=ax[0], cbar=False, vmin=0, vmax=1)
print("Размеры массива:", array.size())

array_padded = F.pad(array, pad=[4, 4])
sns.heatmap(array_padded, annot=True, fmt="d", ax=ax[1], cbar=False)
print("Размеры массива с padding:", array.size())

### Transpose convolution

Способы восстановления пространственных размерностей которые мы рассмотрели, не содержали обучаемых параметров.


Обычная свертка:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-16.png" width="700">


Upsample/transpose convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-17.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-15.png" width="700">

[Блог-пост про 2d свертки с помощью пеермножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Документация к ConvTranspose2d](
https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-19.png" width="700">



In [ ]:
import torch
from torch import nn
import seaborn as sns

# exact output size can be also specified as an argument
input = torch.randn(1, 16, 16, 16)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
# ...
with torch.no_grad():
    upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
    h = downsample(input)
    print("Downsampled size", h.size())

    output = upsample(h, output_size=input.size())
    print("Upsampled size", output.size())

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
sns.heatmap(input[0, 0, :, :], ax=ax[0], cbar=False, vmin=-2, vmax=2)
ax[0].set_title("Input")
sns.heatmap(h[0, 0, :, :], ax=ax[1], cbar=False, vmin=-2, vmax=2)
ax[1].set_title("Downsampled")
sns.heatmap(output[0, 0, :, :], ax=ax[2], cbar=False, vmin=-2, vmax=2)
ax[2].set_title("Upsampled")

## U-Net: Convolutional Networks for Biomedical Image Segmentation

Популярная архитектура для сегментации. Изначально была предложена ([оригинальная статья](https://arxiv.org/abs/1505.04597)) для анализа  медицинских изображений.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-20.png" width="700">

[Реализация на PyTorch](https://github.com/milesial/Pytorch-UNet)

[U-Net на PyTorch Hub](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/)

[Статья-разбор](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)


Обратите внимание на серые стрелки на схеме ....



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">

При конкатенации пространственные размеры должны совпадать.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-22.png" width="700">

После upsample блоков ReLU не используется.


## Несколько классов

Если требуется классифицировать несколько класов объектов, то выходной слой нужно изменить соответствующим образом.

Разберем несколько архитектур для мультиклассовой сегментации

###FCN 

Fully Convolutional Network
для токо что бы не было путаницы с Fully Connected Network
последние именуют MLP (Multi Layer Perceptron)

[Пример реализации 1](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/)

[Пример реализации 2](https://pytorch.org/vision/stable/models.html#semantic-segmentation)

Предобученная модель была обучена на части датасета COCO train2017 (на 20 категориях, представленых так же в датасете  Pascal VOC). Использовались следубщие классы:

`['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']`

In [ ]:
import torchvision
from PIL import Image
from torchvision import transforms
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classes = [
    "__background__",
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor",
]

fcn_model = torchvision.models.segmentation.fcn_resnet50(
    pretrained=True, num_classes=21
)

preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = preprocess(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))  # ['out'][0]

Возвращаются 2 массива

* out - at each location, there are unnormalized probabilities corresponding to the prediction of each class

* aux - contains the auxillary loss values per-pixel. In inference mode, output['aux'] is not usefulcontains the auxillary loss values per-pixel. In inference mode, output['aux'] is not useful

In [ ]:
print(output.keys())  # Ordered dictionary
print("out", output["out"].shape, "Batch, class_num, h, w")
print("aux", output["aux"].shape, "Batch, class_num, h, w")
# aux and output['aux'] contains the auxillary loss values per-pixel. In inference mode, output['aux'] is not usefulcontains the auxillary loss values per-pixel. In inference mode, output['aux'] is not useful

# at each location, there are unnormalized probabilities corresponding to the prediction of each class
output_predictions = output["out"][0].argmax(0)  # for first element of batch
print(output_predictions.shape)

fig = plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

indexes = output_predictions
fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(5):
        mask = torch.zeros(indexes.shape)
        mask[indexes == i] = 255
        # if mask.max() > 0:
        ax[row, col].set_title(classes[i])
        ax[row, col].imshow(mask)
        i += 1

### DeepLab


[Реализация на PyTorch](https://pytorch.org/vision/stable/models.html#deeplabv3)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-24.png" width="700">

### Spatial pyramid pooling (SPP) layer


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-23.png" width="700">

В pytorch нет отдельного модуля, но результат может быть получени применением нескольких AdaptiveMaxPool2d с разными размерами выходов ...

### Atros (Dilated) Convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-26.png" width="700">

In [ ]:
from torch import nn
import torch

# Atros example
with torch.no_grad():
    input = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)
    print("Input shape:", input.size())

    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=1, bias=False)

    conv.weight = nn.Parameter(torch.tensor([[[[2, 2], [2, 2]]]], dtype=torch.float))

    out = conv(input.unsqueeze(0))
    print("Output:", out)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :],
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 1")

In [ ]:
conv = nn.Conv2d(
    1, 1, kernel_size=2, dilation=2, bias=False
)  # Fell free to change dilation

conv.weight = nn.Parameter(torch.tensor([[[[2, 2], [2, 2]]]], dtype=torch.float))

out = conv(input.unsqueeze(0))
print(out)
print(out.shape)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 2")

In [ ]:
input = torch.tensor([[[0, 1, 0], [1, 1, 1], [0, 1, 0]]], dtype=torch.float)
out = conv(input.unsqueeze(0))
print(out)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 2")

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-27.png" width="700">

# Обучение

Входные данные
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-28.png" width="700">

### IoU - ценка точности


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="900">

### Pixel-wise cross entropy loss


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-29.png" width="700">

In [ ]:
from torch import nn

one_class_out = torch.randn(1, 1, 32, 32)
one_class_target = torch.randn(1, 1, 32, 32)

# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
cross_entropy = nn.CrossEntropyLoss()

# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
bce_loss = nn.BCEWithLogitsLoss()
loss = bce_loss(one_class_out, one_class_target)
print("BCE", loss)


two_class_out = torch.randn(1, 2, 32, 32)
two_class_target = torch.randint(1, (1, 32, 32))

print(two_class_out.shape)
print(two_class_target.shape)

loss = cross_entropy(two_class_out, two_class_target)

print("Cross entropy loss", loss)

### DiceLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/dice.jpeg" width="700">

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
from torch import nn


class BinaryDiceLoss(nn.Module):
    """Soft Dice loss of binary class
    Args:
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
       Returns:
        Loss tensor

    """

    def __init__(self, p=2, epsilon=1e-6):
        super().__init__()
        self.p = p  # pow degree
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        # https://pytorch.org/docs/stable/generated/torch.mul.html
        num = torch.sum(torch.mul(predict, target), dim=1) + self.epsilon
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.epsilon
        loss = 1 - 2 * num / den

        return loss.mean()  # over batch


criterion = BinaryDiceLoss()
output = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)


target = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)

soft_loss = criterion(output.unsqueeze(0), target.unsqueeze(0))
print("Loss", soft_loss)

# Детекстирование (Object detection)



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">

Постановка задачи:

- опредлить координаты прямоугольника в котором заключен объект.

Пример из задания для семинара:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-05.png" width="700">

Задачу семантической сегментации мы решали через классификацию. Для детектирования разумно использовать регрессию.

В случае для одного объекта можно обучить модель предсказывать числа:

* координаты центра + ширину и высоту
* координаты правого верхнего и левого нижнего углов
* координаты вершин полигона ...



In [ ]:
from torchvision.models import resnet18
from torch import nn
import torch

# Возьмем обученную сеть
resnet_detector = resnet18(pretrained=True)

# Заменим голову на предсказание 4х точек (x1,y1 x2,y2)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features, 4)  # x1,y1 x2,y2

criterion = nn.MSELoss()

# Это случайный пример. Не ожидайте результатов
input = torch.rand((1, 3, 224, 224))
target = torch.tensor([[0.1, 0.1, 0.5, 0.5]])  # x1,y1 x2,y2 or x,y w,h
output = resnet_detector(input)
loss = criterion(output, target)
print(output)
print("Loss:", loss)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-07.png" width="700">


На прошлом семинаре мы упоминали про модели которые ищут ключевые точки на лице человека (MTCNN). Можно использовать тот же подход для поиска любых точек.


Начнем с ситуации когда объект один.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-08.png" width="500">





У нас есть предобученный классификатор.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-09.png" width="700">

и мы хотим научить его определять местоположение объекта.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-10.png" width="700">

Для этого можно использовать две лосс функции: одна - будет оценивать ошибку классификации, другая - локализации.

Результаты потребуется каким-то образом объединить. В простейшем случае - сложить.

 ## Regression loss

$\mathrm{MSE} = \frac{\sum^n_{i=1}(y_i-y_i^p)^2}{n}$ - L2/ MSE/ Mean Square Error/ Среднеквадратичная ошибка 


$\mathrm{MAE} = \frac{\sum^n_{i=1}|y_i-y_i^p|}{n}$ - L1/ MAE/ Mean Absolute Error/ Средняя ошибка


$\
    L_{\delta}(y, f(x))=\left\{
                \begin{array}{ll}
                  \frac{1}{2}(y-f(x))^2 \qquad \mathrm{for } |y-f(x)| \leq \delta\\
                  \delta|y-f(x)|-\frac{1}{2}\delta^2 \qquad \mathrm{otherwise}
                \end{array}
              \right.
  $ - Huber Loss/ Smooth Mean ABsolute Error/ Функция потерь Хьюбера


$L(y,y^p)=\sum_{i=1}^n log(cosh(y^p_i-y_i))$ - Log-Cosh Loss/ Логарифм гиперболического косинуса


 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-12.png" width="700">

MAE vs MSE

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-13.png" width="700">

Huber vs Log-cos



## Multitask loss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-14.png" width="500">


[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)

[Пример реализации MultiTask learning](https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb)


## Детектирование нескольких объектов

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-042.gif" width="700">



### **a) Наивный способ решения: скользящее окно**

Перебрать все возможные местопоположения объектов и классифицировать фрагменты.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-044.gif" width="700">

- очень долго


### **b) Эвристика**

 Выбрать области в которых вероятность нахождения объекта наиболее высока (ROI = regions of interest) и проводить поиск только в них.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-049.png" width="700">



### Selective search

Один из таких алгоритмов:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-18.png" width="1000">

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

Возвращает порядка 2000 прямоугольников для изображения.

С таким количеством уже можно работать ...

### R-CNN - Region CNN 
Построенна по такому принципу:

- на изображении ищутся ROI 
- для кажого делается resize 
- каждый ROI обрабатывается сверточной сетью
которая предсказывает класс кобъекта который в него попал

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-055.png" width="700">

Кроме класса модель предсказывает смещения для каждого bounding box

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-20.png" width="700">

### NMS

Теперь возникает другая проблема в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box) которые частично прекрывают друг друга.

Что бы избавиться от них используется другой алгоритм
Non maxima suppression

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-22.png" width="700">


Его задача избавиться об bbox которые накладиваются на истинный

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-23.png" width="700">




Для оценки схожести обычно используется метрика IoU
а заначение IoU при котором bbox считаются принадлежащими одному объекту является гиперпараметром (часто 0.5)

Soft NMS
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-25.png" width="700">

### Fast R-CNN

Проблеммой описанного выше подхода является скорость.
Так как мы вынужденны применять CNN порядка 2000 раз (в зависимости от эвристики которая генерирует ROI)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-26.png" width="700">









И решением является поиск ROI не на самом изображении, а на карте признаков, полученной после обработки всего изображения CNN. В таком случае большая часть сверток выполняется только один раз.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-27.png" width="700">

Это радикально ускоряет процесс

### ROI Pooling

Появляется новая задача - 'resize' ROI на карте признаков.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-28.png" width="700">

[Документация Roi Pooling](https://pytorch.org/docs/stable/torchvision/ops.html#torchvision.ops.roi_pool)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-29.png" width="700">






<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-077.png" width="700">

Скорость работы CNN снизилась и теперь узким местом становится эвристика для поиска ROI

### Faster R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-078.png" width="700">

Идеz: пусть сеть сама предсказывает ROI по карте признаков

Для обучения требуется посчитать 4 loss.

### Region proposal network

Карта признаков имеет фиксированные и относительно небольшие пространственные размеры (например 20x15)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-34.png" width="700">



Поэтому можно вернуться к идее скользящего окна которая была отвергнута в самом начале.

При этом можно использовать окна нескольких форм

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-35.png" width="700">

Предсказываются два значения:

* вероятность того что в ROI находится объект
* смещения





Сама сеть при этом может быть очень простой:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/rpn.jpeg" width="700">



В результате скорость увеличивается почте в 10 раз

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-086.png" width="700">

[Модель на PyTorch](https://pytorch.org/vision/stable/models.html#faster-r-cnn)

In [ ]:
import torchvision

fr_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True, progress=True, num_classes=91, pretrained_backbone=True
)
fr_rcnn.eval()

In [ ]:
catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(
    imgIds[12]
)  # http://images.cocodataset.org/val2017/000000370208.jpg
img = img_list[0]
print("Image data", img)

plt.figure(figsize=(10, 10))
pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

In [ ]:
import torchvision.transforms.functional as TF
from PIL import ImageDraw
import matplotlib.pyplot as plt

with torch.no_grad():
    tensor = TF.pil_to_tensor(pil_img) / 255
    output = fr_rcnn(tensor.unsqueeze(0))
    #   print(output[0].keys())
    #   print(output[0]['boxes'])
    #   print(output[0]['labels'])
    #   print(output[0]['scores'])

    draw = ImageDraw.Draw(pil_img)

    for i, bbox in enumerate(output[0]["boxes"]):
        if output[0]["scores"][i] > 0.5:
            draw.rectangle((tuple(bbox[:2].numpy()), tuple(bbox[2:].numpy())), width=2)
    plt.figure(figsize=(10, 10))
    plt.imshow(pil_img)
    plt.show()

### Two stage detector

Faster RCNN == Two stage detector

<img src ="http://edunet.kea.su/repo/src/
L12_Segmentation_Detection/img/lecture_12-089.png" width="700">

На среднем и верхнем слое выполняются очень похожие операции. Разница в том что на последнем слое предсказывается класс объекта, а промежуточном только вероятность его присутствия (objectness)

### One Stage detector

Если сразу предсказывать класс, то можно избавиться от второй стадии.

<img src ="https://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-1.png" width="700">


Детекторы работающие "за один проход":

YOLO, SSD, RetinaNet

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-2.png" width="700">

[Сравнение скорости моделей](https://pytorch.org/vision/stable/models.html#runtime-characteristics)

#### [SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/ssd.png" width="700">

* модель VGG-16, предобученная на ImageNet
* manually defines a collection of aspect ratios to use for the B bounding boxes at each grid cell + offsets (x,y,w,h)
* напрямую предсказывает вероятность того, что класс присутствует в bounding box.
* есть класс для "background"

#### Retina Net - [Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)



#### FocalLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-43.png" width="1000">

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

####Feature pyramyd network

[Feature Pyramid Networks for Object Detection](https://arxiv.org/pdf/1612.03144.pdf)

Это feature extractor для детекторов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">


Так же как и в случае с сегментацией, точность повышается если делать предсказания на картах признаков разных масштабов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn1.jpeg" width="400">



Но первые слои содержат мало семантической информации (только низкоуровневые признаки). Из за этого детектирование(и сегментация) мелких объектов удается хуже.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn2.png" width="400">


идея состоит в том что бы делать предсказание с учетом семантической информации полученной на более глубоких слоях. 


При этом признаки суммируются.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn3.png" width="700">




Затем к новым картам признаков может применяться дополнительная свертка.


На выходе получаем карты признаков P2 - P5 на которых уже предсказываются bounding box.


В случае 2-stage детектора (RCNN) карты подаются на вход RPN 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn5.jpeg" width="700">


А признаки для предсказаний используются из backbone 
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn6.jpeg" width="700">



RetinaNet использует выходы FPN и для предсказаний класса и bbox. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-42.png" width="700">


[Блог-пост про FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)



###YOLO

* [2016 You Only Look Once: Unified, Real-Time Object Detection.](https://arxiv.org/pdf/1506.02640.pdf) 
* [2017 YOLO9000: Better, Faster, Stronger.](https://arxiv.org/pdf/1506.02640.pdf)
* [2018 YOLOv3: An Incremental Improvement](https://arxiv.org/pdf/1804.02767.pdf)

Joseph Redmon

* April 2020, Alexey Bochkovskiy  “[YOLOv4: Optimal Speed and Accuracy of Object Detection"](https://arxiv.org/abs/2004.10934)
* 9 June 2020 [YOLOv5 Glenn Jocher](https://github.com/ultralytics/yolov5)

Первые версии проигрывали конкурентам. Но проект развивался. В настоящий момент это пожалуй оптимальный детектор по соотношению качество разпознавания/скорость.


YOLOv3

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov3.jpeg" width="700">

YOLOv4

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov4.jpeg" width="700">
- SPP block
- Dense Block
- Больше слоев
- online Augmentation


YOLOv5


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-47.png" width="700">

Статья не публиковалась.
Точность сравнима  с v4 но модель определенно лучше упакованна.

In [ ]:
catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[5])
img = img_list[0]
print("Image data", img)

pil_img = coco2pil(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

Загрузка модели с Torch Hub



In [ ]:
import torch

# Load model from torch
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)

Из коробки работает с изображениями в разных форматах и даже url, автоматически меняет размер входного изображения, возвращает объект с результатами ...

In [ ]:
from PIL import Image

# Apply yolov5 model
results = model(pil_img)
results.print()
results.save()  # image on disk

print(type(results.xyxy), len(results.xyxy), results.xyxy[0].shape)
results.pandas().xyxy[0]

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# annIds = coco.getAnnIds(imgIds=[448263])
# anns = coco.loadAnns(annIds)

cv_img = np.array(pil_img)
RGB_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

annos = results.pandas().xyxy[0]

for i in range(len(annos)):
    x_min, y_min, x_max, y_max = (
        int(annos["xmin"].iloc[i]),
        int(annos["ymin"].iloc[i]),
        int(annos["xmax"].iloc[i]),
        int(annos["ymax"].iloc[i]),
    )
    # width, height = int(annos['xmax']-annos['xmin']), int(annos['ymax']-annos['ymin'])
    if annos["name"].iloc[i] == "person":
        color = (255, 255, 255)
    if annos["name"].iloc[i] == "bicycle":
        color = (0, 0, 255)
    if annos["name"].iloc[i] == "backpack":
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x_min, y_min), (x_max, y_max), color, 2)
cv2_imshow(RGB_img)

Однако если подать на вход модели тензор, выходы радикально меняются. YOLO переходит в режим обучения, что довольно не очевидно

In [ ]:
dummy_input = torch.rand((1, 3, 416, 416))
results = model(dummy_input)
print(type(results), len(results))
print(results[0].shape)
print(type(results[1]), len(results[1]))
for e in results[1]:
    print(e.shape)
# print(results[0], results[1].shape) # list of two elements

## Нard Example Mining

Представим что камера видеонаблюдения установленна на улице.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-48.png" width="700">

Целевые объекты, могут появляться достаточно редко (особено ночью)
Но на каждом из кадров будет фон, который будет сильно меняться в зависимости от освещения погодных условий и.т.п.

В результате возникнут изображения с образами которых не было в датасетах и они приведут к ложным срабатываниям.

Что бы дообучить сеть можно сохранять кадры с такими срабатываниями и добавлять их в датасет.



Online hard example mining

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-49.png" width="700">

Можно делать это непосредственно при обучении во время формирования batch-а

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

Или даже заложить в структуру модели:

[Training Region-based Object Detectors with Online Hard Example Mining](https://arxiv.org/pdf/1604.03540.pdf)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors](https://arxiv.org/pdf/1804.04606.pdf)

# Оценка точности

## COCO mAP

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-51.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

РАЗБИРАЕМ КАК СЧИТАЕТСЯ mAP

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/map_data.jpeg" width="700">



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/map.jpeg" width="1000">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-54.png" width="700">




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-56.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-57.png" width="1000">

[Конвертация результатов сегментации в COCO формат](https://www.javaer101.com/en/article/18652684.html).



Синхронизируем метки Pascal2Coco

In [ ]:
pascal2coco = {}
print(classes)


def find_in_dic(dic, val):
    for key in dic.keys():
        if dic.get(key, None) == val:
            return key
    return 0  # Assign missed classes to bg


print(num2cat)
for i in range(1, len(classes)):  # Skip BG
    # if cats.get()
    coco_ind = find_in_dic(num2cat, classes[i])
    pascal2coco[i] = coco_ind

print(pascal2coco)

In [ ]:
img = coco.loadImgs(448263)[0]
print(img)
annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)

In [ ]:
from pycocotools import mask
import numpy as np
from itertools import groupby
import json


def binary_mask_to_rle(binary_mask):
    rle = {"counts": [], "size": list(binary_mask.shape)}
    counts = rle.get("counts")
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order="F"))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle


detection_res = []
for i, cls_name in enumerate(classes):
    binary_mask = torch.zeros(indexes.shape)
    binary_mask[indexes == i] = 1  # Form FCN for baseball

    if i > 0 and torch.max(binary_mask) > 0:
        uncompressed_rle = binary_mask_to_rle(binary_mask.numpy())  # encoded_gt,
        fortran_gt_binary_mask = np.asfortranarray(binary_mask).astype("uint8")
        encoded_gt = mask.encode(fortran_gt_binary_mask)
        bbox = list(mask.toBbox(encoded_gt))
        print(bbox)

        detection_res.append(
            {
                "score": 1.0,  # dummy
                "category_id": pascal2coco[i],
                "segmentation": uncompressed_rle,
                "bbox": bbox,
                "image_id": 448263,
                "iscrowd": 0,
            }
        )

print(detection_res)
with open("seg_res.json", "w", encoding="utf-8") as f:
    json.dump(detection_res, f, ensure_ascii=False, indent=4)

plt.subplot(1, 2, 1)
pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
plt.imshow(pil_img)
coco.showAnns(anns, draw_bbox=True)
plt.title("Annotation from COCO")

plt.subplot(1, 2, 2)
plt.imshow(pil_img)
coco.showAnns(detection_res, draw_bbox=True)
plt.title("Detection")

В предыдущем шаге мы посчитали предсказаниее от YOLO. Давайте оценим его точность

In [ ]:
import json

with open("seg_gt.json", "w", encoding="utf-8") as f:
    json.dump(anns, f, ensure_ascii=False, indent=4)

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# loadRes will generate a new COCO type instance based on coco_gt and return
coco_dt = coco.loadRes("seg_res.json")
coco_gt = coco.loadRes("seg_gt.json")

cocoEval = COCOeval(coco_gt, coco_dt, "bbox")  # 'segm', 'bbox'
# cocoEval.params.useSegm = True
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

print(cocoEval.stats)

# Instance Segmentation

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-03.png" width="700">

[COCO panoptic](https://cocodataset.org/#panoptic-2020)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-096.png" width="700">







### ROI Align




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-30.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-31.png" width="700">

[Модель Mask R-CNN](https://pytorch.org/vision/stable/models.html#mask-r-cnn)

[Пример запуска Mask R-CNN есть в документации Pytorch](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)



# Список использованной литературы
[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

[Видео-разбор Run-Length Encoding](https://www.youtube.com/watch?v=h6s61a_pqfM)

[Блог-пост про 2d свертки с помощью пеермножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Статья U-Net](https://arxiv.org/abs/1505.04597)
[Блог-пост про U-Net](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

[SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)

[Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

[Feature Pyramid Networks for Object Detection](https://arxiv.org/pdf/1612.03144.pdf)

[2016 You Only Look Once: Unified, Real-Time Object Detection.](https://arxiv.org/pdf/1506.02640.pdf) 

[2017 YOLO9000: Better, Faster, Stronger.](https://arxiv.org/pdf/1506.02640.pdf)

[2018 YOLOv3: An Incremental Improvement](https://arxiv.org/pdf/1804.02767.pdf)

[YOLOv4: Optimal Speed and Accuracy of Object Detection](https://arxiv.org/abs/2004.10934)

[YOLOv5 Glenn Jocher](https://github.com/ultralytics/yolov5)

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining](https://arxiv.org/pdf/1604.03540.pdf)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors](https://arxiv.org/pdf/1804.04606.pdf)